# Deep Convolutional Generative Adversarial Networks with TensorFlow

 In this tutorial, we will try to build a GAN that is able to generate human faces with TensorFlow. Sounds scary, doesn’t it?


## Setup

A helper function will automatically download the CelebA dataset to get you up and running quickly. Be sure to have matplotlib installed to actually see the images.

In [1]:
import helper

data_dir = './data'
# Let's download the dataset
helper.download_celeb_a()

OSError: [Errno 28] No space left on device: 'data/img_align_celeba/032767.jpg'

## The CelebA Dataset

The CelebFaces Attributes Dataset dataset contains over 200,000 celebrity images each with 40 attribute annotations. At this point, we are also going to define to function for batch generation. This function will load our images and give us an array of images according to a batch size we are going to set later. To get some better results, we will crop the images, so that only the faces are showing. We will also normalize the images so that their values are in a range from -0,5 to +0,5. At last, we are going to downscale the images to 56x56 after that.

In [ ]:
%matplotlib inline
import os
from glob import glob
from matplotlib import pyplot
from PIL import Image

def get_image(image_path, width, height, mode):
    """
    Read image from image_path
    """
    image = Image.open(image_path)

    if image.size != (width, height):
        # Remove most pixels that aren't part of a face
        face_width = face_height = 108
        j = (image.size[0] - face_width) // 2
        i = (image.size[1] - face_height) // 2
        image = image.crop([j, i, j + face_width, i + face_height])
        image = image.resize([width, height], Image.BILINEAR)

    return np.array(image.convert(mode))

def get_batch(image_files, width, height, mode='RGB'):
    """
    Get a single image
    """
    data_batch = np.array(
        [get_image(sample_file, width, height, mode) for sample_file in image_files]).astype(np.float32)

    # Make sure the images are in 4 dimensions
    if len(data_batch.shape) < 4:
        data_batch = data_batch.reshape(data_batch.shape + (1,))

    return data_batch

def get_batches(self, batch_size):
    """
    Generate batches
    """
    IMAGE_MAX_VALUE = 255
    IMAGE_WIDTH = 56
    IMAGE_HEIGHT = 56
    shape = len(data_files), IMAGE_WIDTH, IMAGE_HEIGHT, image_channels
    data_files = glob(os.path.join(data_dir, 'img_align_celeba/*.jpg'))

    current_index = 0
    while current_index + batch_size <= shape[0]:
        data_batch = get_batch(
            data_files[current_index:current_index + batch_size],
            *shape[1:3])

        current_index += batch_size

        yield data_batch / IMAGE_MAX_VALUE - 0.5
        

test_index = 500
test_image = get_batch(glob(os.path.join(data_dir, 'img_align_celeba/*.jpg'))[test_index], 56, 56, 'RGB')
pyplot.imshow(test_image)

## Defining network input

Before we can start defining our two networks, we are going to define our inputs. We are doing this to not clutter the training function any more than it already is. Here, we are simply defining TensorFlow Placeholders for our real and fake inputs and for the learning rate.

## The discriminator network

The discriminator is the “art critic”, who tries to distinguish between real and fake images. Simply said, this is a convolutional neural network for image classification. The discriminator network consists of four convolutional layers. For every layer of the network, we are going to perform a convolution, then we are going to perform batch normalization to make the network faster and more accurate and finally, we are going to perform a Leaky ReLu.

## The generator network

The generator goes the other way: It is the artist who is trying to fool the discriminator. This network consists of four deconvolutional layers. In here, we are doing the same as in the discriminator, just in the other direction. First, we take our input, called Z, and feed it into our first deconvolutional layer. Each deconvolutional layer performs a deconvolution and then performs batch normalization and a leaky ReLu as well. Then, we return the tanh activation function.

## Loss Functions

Rather than just having a single loss function, we need to define three: The loss of the generator, the loss of the discriminator when using real images and the loss of the discriminator when using fake images. The sum of the fake image and real image loss is the overall discriminator loss.

## Optimization

Rather than just having a single loss function, we need to define three: The loss of the generator, the loss of the discriminator when using real images and the loss of the discriminator when using fake images. The sum of the fake image and real image loss is the overall discriminator loss.

## Visualization

In the last step of our preparation, we are writing a small helper function to display the generated images in the notebook for us, using the matplotlib library.

## Training

Now, we just get our inputs, losses and optimizers which we defined before, call a TensorFlow session and run it batch per batch. Every 200 steps we are printing out the current progress by showing the generated image and loss. Now lean back and see the faces show up slowly but steady - and we mean slowly but steady! This progress can take up some hours based on your setup.